In [1]:
import os
import json
import numpy as np
import pandas as pd
import pyproj
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib import cm
from pysheds.grid import Grid
from matplotlib import colors
import seaborn as sns
import warnings
from swmm import SwmmIngester
from swmmtoolbox import swmmtoolbox

warnings.filterwarnings('ignore')
sns.set_palette('husl', 8)
sns.set()

%matplotlib inline

In [2]:
dirmap = (64,  128,  1,   2,    4,   8,    16,  32)
grid = Grid.from_raster('../data/n30w100_con', data_name='dem', input_type='ascii')
grid.read_raster('../data/n30w100_dir', data_name='dir', input_type='ascii',
                 data_attrs={'dirmap' : dirmap})
# Specify pour point
x, y = -97.294167, 32.73750
# Delineate the catchment
grid.catchment(x, y, data='dir', out_name='catch',
            recursionlimit=15000, xytype='label', dirmap=dirmap)
# Clip the bounding box to the catchment
grid.clip_to('catch', pad=(1,1,1,1))
grid.accumulation(data='catch', out_name='acc', dirmap=dirmap)
grid.read_raster('../../../Data/GIS/nlcd_2011_landcover_2011_edition_2014_10_10/nlcd_2011_landcover_2011_edition_2014_10_10.img',
                 data_name='cover', window=grid.bbox, window_crs=grid.crs)
grid.read_raster('../../../Data/GIS/nlcd_2011_impervious_2011_edition_2014_10_10/nlcd_2011_impervious_2011_edition_2014_10_10.img',
                 data_name='terrain', window=grid.bbox, window_crs=grid.crs)
cover = grid.view('cover', nodata=0)
ter = grid.view('terrain', nodata=0)
projection = pyproj.Proj('+init=epsg:3083')

In [3]:
initialize = False

In [4]:
intensities = [11.2, 16.87, 23.38]

# Uncontrolled

In [5]:
if initialize:
    if os.path.exists('../data/out/uncontrolled.out'):
        filename = '../data/out/uncontrolled.out'
        c = swmmtoolbox.catalog(filename)
        c = pd.DataFrame(c)
        # Get nodes only
        item_names = c[c[0] == 'node'][1].values
        items = pd.Series(np.repeat(item_names, 2)).astype(str)
        # Get depth head and inflow
        var_indices = pd.Series(np.tile([0, 4], len(item_names))).astype(str)
        item_types = pd.Series(np.repeat('node', len(items))).astype(str)
        # Construct label strings
        labels = (item_types + ',' + items + ',' + var_indices).values.tolist()
        # Extract data
        result = swmmtoolbox.fast_extract(filename, *labels)
        depths = (result[result.columns[pd.Series(result.columns)
                                       .str.endswith('Depth_above_invert')]]
                  .iloc[-1])
        depths.index = pd.Series(depths.index).str.extract('node_(.+)_Depth').values
        item_names = c[c[0] == 'link'][1].values
        items = pd.Series(np.repeat(item_names, 2)).astype(str)
        # Get depth head and inflow
        var_indices = pd.Series(np.tile([0, 1], len(item_names))).astype(str)
        item_types = pd.Series(np.repeat('link', len(items))).astype(str)
        # Construct label strings
        labels = (item_types + ',' + items + ',' + var_indices).values.tolist()
        flows = (result[result.columns[pd.Series(result.columns)
                                       .str.endswith('Flow_rate')]]
                  .iloc[-1])
        flows.index = pd.Series(flows.index).str.extract('link_(.+)_Flow').values
        # Extract data
        result = swmmtoolbox.fast_extract(filename, *labels)
        swmm = SwmmIngester(grid, 'dem', 'dir', 'catch', 'acc',
                            projection, control=True, initialize=True,
                            node_depths=depths, link_flows=flows)
else:
    swmm = SwmmIngester(grid, 'dem', 'dir', 'catch', 'acc',
                        projection, ter, cover, control=False, initialize=False,
                        outlet=472, into_outlet=644)
ixes = []
for intensity in intensities:
    swmm.run_swmm_ingester('../data/inp/uncontrolled_{0}mm.inp'.format(intensity),
                           ixes=ixes, outlet=472, into_outlet=644, intensity=intensity)

# Experiments

# Phi 50

In [6]:
if initialize:
    filename = '../data/out/uncontrolled.out'
    c = swmmtoolbox.catalog(filename)
    c = pd.DataFrame(c)
    # Get nodes only
    item_names = c[c[0] == 'node'][1].values
    items = pd.Series(np.repeat(item_names, 2)).astype(str)
    # Get depth head and inflow
    var_indices = pd.Series(np.tile([0, 4], len(item_names))).astype(str)
    item_types = pd.Series(np.repeat('node', len(items))).astype(str)
    # Construct label strings
    labels = (item_types + ',' + items + ',' + var_indices).values.tolist()

    # Extract data
    result = swmmtoolbox.fast_extract(filename, *labels)

    depths = (result[result.columns[pd.Series(result.columns)
                                   .str.endswith('Depth_above_invert')]]
              .iloc[-1])
    depths.index = pd.Series(depths.index).str.extract('node_(.+)_Depth').values

    filename = '../data/out/uncontrolled.out'
    c = swmmtoolbox.catalog(filename)
    c = pd.DataFrame(c)
    # Get nodes only
    item_names = c[c[0] == 'link'][1].values
    items = pd.Series(np.repeat(item_names, 2)).astype(str)
    # Get depth head and inflow
    var_indices = pd.Series(np.tile([0, 1], len(item_names))).astype(str)
    item_types = pd.Series(np.repeat('link', len(items))).astype(str)
    # Construct label strings
    labels = (item_types + ',' + items + ',' + var_indices).values.tolist()

    # Extract data
    result = swmmtoolbox.fast_extract(filename, *labels)
    
    flows = (result[result.columns[pd.Series(result.columns)
                                   .str.endswith('Flow_rate')]]
              .iloc[-1])
    flows.index = pd.Series(flows.index).str.extract('link_(.+)_Flow').values
else:
    depths = None
    flows = None

In [7]:
with open('../data/experiments_differentiated_50pct_phi50.json', 'r') as infile:
    experiments = json.load(infile)

In [8]:
swmm = SwmmIngester(grid, 'dem', 'dir', 'catch', 'acc',
                    projection, ter, cover, control=True, initialize=initialize,
                    node_depths=depths, link_flows=flows)
for intensity in intensities:
    for experiment in experiments:
        if not '{0}_{1}mm.inp'.format(experiment, intensity) in os.listdir('../data/inp'):
            ixes = experiments[experiment]['controller_locs']
            try:
                swmm.run_swmm_ingester('../data/inp/{0}_{1}mm.inp'.format(experiment, intensity),
                                       ixes=ixes, outlet=472, into_outlet=644, intensity=intensity)
            except:
                print('Error with {0}'.format(experiment))
                continue

# Phi 10

In [6]:
if initialize:
    filename = '../data/out/uncontrolled.out'
    c = swmmtoolbox.catalog(filename)
    c = pd.DataFrame(c)
    # Get nodes only
    item_names = c[c[0] == 'node'][1].values
    items = pd.Series(np.repeat(item_names, 2)).astype(str)
    # Get depth head and inflow
    var_indices = pd.Series(np.tile([0, 4], len(item_names))).astype(str)
    item_types = pd.Series(np.repeat('node', len(items))).astype(str)
    # Construct label strings
    labels = (item_types + ',' + items + ',' + var_indices).values.tolist()

    # Extract data
    result = swmmtoolbox.fast_extract(filename, *labels)

    depths = (result[result.columns[pd.Series(result.columns)
                                   .str.endswith('Depth_above_invert')]]
              .iloc[-1])
    depths.index = pd.Series(depths.index).str.extract('node_(.+)_Depth').values

    filename = '../data/out/uncontrolled.out'
    c = swmmtoolbox.catalog(filename)
    c = pd.DataFrame(c)
    # Get nodes only
    item_names = c[c[0] == 'link'][1].values
    items = pd.Series(np.repeat(item_names, 2)).astype(str)
    # Get depth head and inflow
    var_indices = pd.Series(np.tile([0, 1], len(item_names))).astype(str)
    item_types = pd.Series(np.repeat('link', len(items))).astype(str)
    # Construct label strings
    labels = (item_types + ',' + items + ',' + var_indices).values.tolist()

    # Extract data
    result = swmmtoolbox.fast_extract(filename, *labels)
    
    flows = (result[result.columns[pd.Series(result.columns)
                                   .str.endswith('Flow_rate')]]
              .iloc[-1])
    flows.index = pd.Series(flows.index).str.extract('link_(.+)_Flow').values
else:
    depths = None
    flows = None

In [7]:
with open('../data/experiments_differentiated_50pct_phi10.json', 'r') as infile:
    experiments = json.load(infile)

In [8]:
swmm = SwmmIngester(grid, 'dem', 'dir', 'catch', 'acc',
                    projection, ter, cover, control=True, initialize=initialize,
                    node_depths=depths, link_flows=flows)
for intensity in intensities:
    for experiment in experiments:
        if not '{0}_{1}mm.inp'.format(experiment, intensity) in os.listdir('../data/inp'):
            ixes = experiments[experiment]['controller_locs']
            try:
                swmm.run_swmm_ingester('../data/inp/{0}_{1}mm.inp'.format(experiment, intensity),
                                       ixes=ixes, outlet=472, into_outlet=644, intensity=intensity)
            except:
                print('Error with {0}'.format(experiment))
                continue

# Phi 25

In [9]:
if initialize:
    filename = '../data/out/uncontrolled.out'
    c = swmmtoolbox.catalog(filename)
    c = pd.DataFrame(c)
    # Get nodes only
    item_names = c[c[0] == 'node'][1].values
    items = pd.Series(np.repeat(item_names, 2)).astype(str)
    # Get depth head and inflow
    var_indices = pd.Series(np.tile([0, 4], len(item_names))).astype(str)
    item_types = pd.Series(np.repeat('node', len(items))).astype(str)
    # Construct label strings
    labels = (item_types + ',' + items + ',' + var_indices).values.tolist()

    # Extract data
    result = swmmtoolbox.fast_extract(filename, *labels)

    depths = (result[result.columns[pd.Series(result.columns)
                                   .str.endswith('Depth_above_invert')]]
              .iloc[-1])
    depths.index = pd.Series(depths.index).str.extract('node_(.+)_Depth').values

    filename = '../data/out/uncontrolled.out'
    c = swmmtoolbox.catalog(filename)
    c = pd.DataFrame(c)
    # Get nodes only
    item_names = c[c[0] == 'link'][1].values
    items = pd.Series(np.repeat(item_names, 2)).astype(str)
    # Get depth head and inflow
    var_indices = pd.Series(np.tile([0, 1], len(item_names))).astype(str)
    item_types = pd.Series(np.repeat('link', len(items))).astype(str)
    # Construct label strings
    labels = (item_types + ',' + items + ',' + var_indices).values.tolist()

    # Extract data
    result = swmmtoolbox.fast_extract(filename, *labels)
    
    flows = (result[result.columns[pd.Series(result.columns)
                                   .str.endswith('Flow_rate')]]
              .iloc[-1])
    flows.index = pd.Series(flows.index).str.extract('link_(.+)_Flow').values
else:
    depths = None
    flows = None

In [10]:
with open('../data/experiments_differentiated_50pct_phi25.json', 'r') as infile:
    experiments = json.load(infile)

In [11]:
swmm = SwmmIngester(grid, 'dem', 'dir', 'catch', 'acc',
                    projection, ter, cover, control=True, initialize=initialize,
                    node_depths=depths, link_flows=flows)
for intensity in intensities:
    for experiment in experiments:
        if not '{0}_{1}mm.inp'.format(experiment, intensity) in os.listdir('../data/inp'):
            ixes = experiments[experiment]['controller_locs']
            try:
                swmm.run_swmm_ingester('../data/inp/{0}_{1}mm.inp'.format(experiment, intensity),
                                       ixes=ixes, outlet=472, into_outlet=644, intensity=intensity)
            except:
                print('Error with {0}'.format(experiment))
                continue

# Phi 100

In [5]:
if initialize:
    filename = '../data/out/uncontrolled.out'
    c = swmmtoolbox.catalog(filename)
    c = pd.DataFrame(c)
    # Get nodes only
    item_names = c[c[0] == 'node'][1].values
    items = pd.Series(np.repeat(item_names, 2)).astype(str)
    # Get depth head and inflow
    var_indices = pd.Series(np.tile([0, 4], len(item_names))).astype(str)
    item_types = pd.Series(np.repeat('node', len(items))).astype(str)
    # Construct label strings
    labels = (item_types + ',' + items + ',' + var_indices).values.tolist()

    # Extract data
    result = swmmtoolbox.fast_extract(filename, *labels)

    depths = (result[result.columns[pd.Series(result.columns)
                                   .str.endswith('Depth_above_invert')]]
              .iloc[-1])
    depths.index = pd.Series(depths.index).str.extract('node_(.+)_Depth').values

    filename = '../data/out/uncontrolled.out'
    c = swmmtoolbox.catalog(filename)
    c = pd.DataFrame(c)
    # Get nodes only
    item_names = c[c[0] == 'link'][1].values
    items = pd.Series(np.repeat(item_names, 2)).astype(str)
    # Get depth head and inflow
    var_indices = pd.Series(np.tile([0, 1], len(item_names))).astype(str)
    item_types = pd.Series(np.repeat('link', len(items))).astype(str)
    # Construct label strings
    labels = (item_types + ',' + items + ',' + var_indices).values.tolist()

    # Extract data
    result = swmmtoolbox.fast_extract(filename, *labels)
    
    flows = (result[result.columns[pd.Series(result.columns)
                                   .str.endswith('Flow_rate')]]
              .iloc[-1])
    flows.index = pd.Series(flows.index).str.extract('link_(.+)_Flow').values
else:
    depths = None
    flows = None

In [6]:
with open('../data/experiments_differentiated_50pct_phi100.json', 'r') as infile:
    experiments = json.load(infile)

In [7]:
swmm = SwmmIngester(grid, 'dem', 'dir', 'catch', 'acc',
                    projection, ter, cover, control=True, initialize=initialize,
                    node_depths=depths, link_flows=flows)
for intensity in intensities:
    for experiment in experiments:
        if not '{0}_{1}mm.inp'.format(experiment, intensity) in os.listdir('../data/inp'):
            ixes = experiments[experiment]['controller_locs']
            try:
                swmm.run_swmm_ingester('../data/inp/{0}_{1}mm.inp'.format(experiment, intensity),
                                       ixes=ixes, outlet=472, into_outlet=644, intensity=intensity)
            except:
                print('Error with {0}'.format(experiment))
                continue

# Phi 50, top open 1%

In [5]:
if initialize:
    filename = '../data/out/uncontrolled.out'
    c = swmmtoolbox.catalog(filename)
    c = pd.DataFrame(c)
    # Get nodes only
    item_names = c[c[0] == 'node'][1].values
    items = pd.Series(np.repeat(item_names, 2)).astype(str)
    # Get depth head and inflow
    var_indices = pd.Series(np.tile([0, 4], len(item_names))).astype(str)
    item_types = pd.Series(np.repeat('node', len(items))).astype(str)
    # Construct label strings
    labels = (item_types + ',' + items + ',' + var_indices).values.tolist()

    # Extract data
    result = swmmtoolbox.fast_extract(filename, *labels)

    depths = (result[result.columns[pd.Series(result.columns)
                                   .str.endswith('Depth_above_invert')]]
              .iloc[-1])
    depths.index = pd.Series(depths.index).str.extract('node_(.+)_Depth').values

    filename = '../data/out/uncontrolled.out'
    c = swmmtoolbox.catalog(filename)
    c = pd.DataFrame(c)
    # Get nodes only
    item_names = c[c[0] == 'link'][1].values
    items = pd.Series(np.repeat(item_names, 2)).astype(str)
    # Get depth head and inflow
    var_indices = pd.Series(np.tile([0, 1], len(item_names))).astype(str)
    item_types = pd.Series(np.repeat('link', len(items))).astype(str)
    # Construct label strings
    labels = (item_types + ',' + items + ',' + var_indices).values.tolist()

    # Extract data
    result = swmmtoolbox.fast_extract(filename, *labels)
    
    flows = (result[result.columns[pd.Series(result.columns)
                                   .str.endswith('Flow_rate')]]
              .iloc[-1])
    flows.index = pd.Series(flows.index).str.extract('link_(.+)_Flow').values
else:
    depths = None
    flows = None

In [6]:
with open('../data/experiments_differentiated_50pct_phi50.json', 'r') as infile:
    experiments = json.load(infile)

In [7]:
swmm = SwmmIngester(grid, 'dem', 'dir', 'catch', 'acc',
                    projection, ter, cover, control=True, initialize=initialize,
                    node_depths=depths, link_flows=flows)
for intensity in intensities:
    for experiment in experiments:
        if not '{0}_over1pct_{1}mm.inp'.format(experiment, intensity) in os.listdir('../data/inp'):
            ixes = experiments[experiment]['controller_locs']
            try:
                swmm.run_swmm_ingester('../data/inp/{0}_over1pct_{1}mm.inp'.format(experiment, intensity),
                                       ixes=ixes, outlet=472, into_outlet=644, intensity=intensity, position='top', frac_open=0.1)
            except:
                print('Error with {0}'.format(experiment))
                continue

# Phi 50, bottom open 10%

In [5]:
if initialize:
    filename = '../data/out/uncontrolled.out'
    c = swmmtoolbox.catalog(filename)
    c = pd.DataFrame(c)
    # Get nodes only
    item_names = c[c[0] == 'node'][1].values
    items = pd.Series(np.repeat(item_names, 2)).astype(str)
    # Get depth head and inflow
    var_indices = pd.Series(np.tile([0, 4], len(item_names))).astype(str)
    item_types = pd.Series(np.repeat('node', len(items))).astype(str)
    # Construct label strings
    labels = (item_types + ',' + items + ',' + var_indices).values.tolist()

    # Extract data
    result = swmmtoolbox.fast_extract(filename, *labels)

    depths = (result[result.columns[pd.Series(result.columns)
                                   .str.endswith('Depth_above_invert')]]
              .iloc[-1])
    depths.index = pd.Series(depths.index).str.extract('node_(.+)_Depth').values

    filename = '../data/out/uncontrolled.out'
    c = swmmtoolbox.catalog(filename)
    c = pd.DataFrame(c)
    # Get nodes only
    item_names = c[c[0] == 'link'][1].values
    items = pd.Series(np.repeat(item_names, 2)).astype(str)
    # Get depth head and inflow
    var_indices = pd.Series(np.tile([0, 1], len(item_names))).astype(str)
    item_types = pd.Series(np.repeat('link', len(items))).astype(str)
    # Construct label strings
    labels = (item_types + ',' + items + ',' + var_indices).values.tolist()

    # Extract data
    result = swmmtoolbox.fast_extract(filename, *labels)
    
    flows = (result[result.columns[pd.Series(result.columns)
                                   .str.endswith('Flow_rate')]]
              .iloc[-1])
    flows.index = pd.Series(flows.index).str.extract('link_(.+)_Flow').values
else:
    depths = None
    flows = None

In [6]:
with open('../data/experiments_differentiated_50pct_phi50.json', 'r') as infile:
    experiments = json.load(infile)

In [7]:
swmm = SwmmIngester(grid, 'dem', 'dir', 'catch', 'acc',
                    projection, ter, cover, control=True, initialize=initialize,
                    node_depths=depths, link_flows=flows)
for intensity in intensities:
    for experiment in experiments:
        if not '{0}_under10pct_{1}mm.inp'.format(experiment, intensity) in os.listdir('../data/inp'):
            ixes = experiments[experiment]['controller_locs']
            try:
                swmm.run_swmm_ingester('../data/inp/{0}_under10pct_{1}mm.inp'.format(experiment, intensity),
                                       ixes=ixes, outlet=472, into_outlet=644, intensity=intensity, position='bottom', frac_open=1)
            except:
                print('Error with {0}'.format(experiment))
                continue